#Task:
Calculate efficiency of the stations based on the following features:
1. Distance from nearest bus station
2. Distance from nearest train station
3. Distance to the 3 nearest divvy stations
4. Supply vs demand of the bikes at the stations

We will add some more features to calculate efficiency after performing some more EDA to see which factors would have an effect on the efficiency.

We are yet to add the calculation for supply and demand as we need some pre processing for that.

#Use case:
To calculate how efficient is the location of each of the stations. We will also use these individual efficiencies to calculate efficiency of different regions of Chicago using the clusters created earlier.

#Analysis:
1. We were able to write an algorithm to calculate efficiency by giving certain weights (importance) to each features.
2. We then scale the efficiency so that we can obtain a value between 0 & 1. We will then decide on a threshold which can determine if a station is efficient or not. (For eg. taking the threshold as 0.5, any station with efficiency greater than 0.5 would be efficient and lesser than 0.5 would be inefficient.)

We realize that the weights that we have assigned are prone to bias. We will be conducting a survey in order to eliminate this bias and find out the importance of these features with respect to the opinion of the general public.

In [1]:
import pandas as pd
import numpy as np
import geopy.distance
from ast import literal_eval

In [2]:
# pip install geopy

In [3]:
stations = pd.read_csv("GetNearestBusAndTrainStation.csv")
stations = stations.drop(columns="Unnamed: 0")
stations

,ID,Station Name,Total Docks,Docks in Service,Status,Latitude,Longitude,Location,nearest_bus_station_name,nearest_bus_station_distance,nearest_train_station_name,nearest_train_station_distance
0,1594046383808271024,Troy St & Jackson Blvd,9,9,In Service,41.877505,-87.704850,"(41.877505, -87.70485)",Jackson & Kedzie,0.081453,Kedzie-Homan,0.365048
1,641,Central Park Ave & Bloomingdale Ave,11,11,In Service,41.914166,-87.716755,"(41.914166, -87.716755)",Armitage & Central Park,0.342105,Logan Square,1.857948
2,1683527931525155814,Public Rack - Cornell Ave & 87th Pl,1,1,In Service,41.736881,-87.583146,"(41.73688127, -87.58314552)",87th Street & East End,0.020426,87th,3.462482
3,367,Racine Ave & 35th St,15,15,In Service,41.830689,-87.656211,"(41.83068856472101, -87.65621066093445)",35th Street & Racine,0.050069,Ashland,1.213685
4,1673852313397164648,Public Rack - Francisco Ave & Touhy Ave,2,2,In Service,42.011865,-87.701317,"(42.011865, -87.701317)",Touhy & Francisco,0.072283,Howard,2.486505
...,...,...,...,...,...,...,...,...,...,...,...,...
1414,1715846705501012844,Public Rack - Lawndale Ave & Roosevelt,2,2,In Service,41.866225,-87.717918,"(41.866225, -87.717918)",Roosevelt & Lawndale,0.005609,Pulaski,1.058657
1415,1715846718385914742,Public Rack - Ashland Ave & 74th St,2,2,In Service,41.759735,-87.663736,"(41.759735, -87.663736)",74th Street & Ashland,0.034132,Ashland/63rd,2.124192
1416,320,Loomis St & Lexington St,19,19,In Service,41.872187,-87.661501,"(41.872187, -87.661501)",Harrison & Loomis,0.217218,Racine,0.447972
1417,54,Ogden Ave & Chicago Ave,19,19,In Service,41.896362,-87.654061,"(41.896362458, -87.6540612729)",Chicago & Milwaukee (Blue Line),0.086847,Chicago,0.100847


In [4]:
for i in stations.index:
    stations["nearest_train_station_distance"][i] = stations["nearest_train_station_distance"][i] * 0.621
    stations["nearest_bus_station_distance"][i] = stations["nearest_bus_station_distance"][i] * 0.621
    
stations

/var/folders/78/q12w2l756hv6nzyfq63_g6980000gn/T/ipykernel_19165/2378682040.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations["nearest_train_station_distance"][i] = stations["nearest_train_station_distance"][i] * 0.621
/var/folders/78/q12w2l756hv6nzyfq63_g6980000gn/T/ipykernel_19165/2378682040.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations["nearest_bus_station_distance"][i] = stations["nearest_bus_station_distance"][i] * 0.621


,ID,Station Name,Total Docks,Docks in Service,Status,Latitude,Longitude,Location,nearest_bus_station_name,nearest_bus_station_distance,nearest_train_station_name,nearest_train_station_distance
0,1594046383808271024,Troy St & Jackson Blvd,9,9,In Service,41.877505,-87.704850,"(41.877505, -87.70485)",Jackson & Kedzie,0.050582,Kedzie-Homan,0.226695
1,641,Central Park Ave & Bloomingdale Ave,11,11,In Service,41.914166,-87.716755,"(41.914166, -87.716755)",Armitage & Central Park,0.212447,Logan Square,1.153785
2,1683527931525155814,Public Rack - Cornell Ave & 87th Pl,1,1,In Service,41.736881,-87.583146,"(41.73688127, -87.58314552)",87th Street & East End,0.012684,87th,2.150201
3,367,Racine Ave & 35th St,15,15,In Service,41.830689,-87.656211,"(41.83068856472101, -87.65621066093445)",35th Street & Racine,0.031093,Ashland,0.753699
4,1673852313397164648,Public Rack - Francisco Ave & Touhy Ave,2,2,In Service,42.011865,-87.701317,"(42.011865, -87.701317)",Touhy & Francisco,0.044888,Howard,1.544120
...,...,...,...,...,...,...,...,...,...,...,...,...
1414,1715846705501012844,Public Rack - Lawndale Ave & Roosevelt,2,2,In Service,41.866225,-87.717918,"(41.866225, -87.717918)",Roosevelt & Lawndale,0.003483,Pulaski,0.657426
1415,1715846718385914742,Public Rack - Ashland Ave & 74th St,2,2,In Service,41.759735,-87.663736,"(41.759735, -87.663736)",74th Street & Ashland,0.021196,Ashland/63rd,1.319123
1416,320,Loomis St & Lexington St,19,19,In Service,41.872187,-87.661501,"(41.872187, -87.661501)",Harrison & Loomis,0.134892,Racine,0.278191
1417,54,Ogden Ave & Chicago Ave,19,19,In Service,41.896362,-87.654061,"(41.896362458, -87.6540612729)",Chicago & Milwaukee (Blue Line),0.053932,Chicago,0.062626


In [5]:
stations["efficiency"] = 0
stations

,ID,Station Name,Total Docks,Docks in Service,Status,Latitude,Longitude,Location,nearest_bus_station_name,nearest_bus_station_distance,nearest_train_station_name,nearest_train_station_distance,efficiency
0,1594046383808271024,Troy St & Jackson Blvd,9,9,In Service,41.877505,-87.704850,"(41.877505, -87.70485)",Jackson & Kedzie,0.050582,Kedzie-Homan,0.226695,0
1,641,Central Park Ave & Bloomingdale Ave,11,11,In Service,41.914166,-87.716755,"(41.914166, -87.716755)",Armitage & Central Park,0.212447,Logan Square,1.153785,0
2,1683527931525155814,Public Rack - Cornell Ave & 87th Pl,1,1,In Service,41.736881,-87.583146,"(41.73688127, -87.58314552)",87th Street & East End,0.012684,87th,2.150201,0
3,367,Racine Ave & 35th St,15,15,In Service,41.830689,-87.656211,"(41.83068856472101, -87.65621066093445)",35th Street & Racine,0.031093,Ashland,0.753699,0
4,1673852313397164648,Public Rack - Francisco Ave & Touhy Ave,2,2,In Service,42.011865,-87.701317,"(42.011865, -87.701317)",Touhy & Francisco,0.044888,Howard,1.544120,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414,1715846705501012844,Public Rack - Lawndale Ave & Roosevelt,2,2,In Service,41.866225,-87.717918,"(41.866225, -87.717918)",Roosevelt & Lawndale,0.003483,Pulaski,0.657426,0
1415,1715846718385914742,Public Rack - Ashland Ave & 74th St,2,2,In Service,41.759735,-87.663736,"(41.759735, -87.663736)",74th Street & Ashland,0.021196,Ashland/63rd,1.319123,0
1416,320,Loomis St & Lexington St,19,19,In Service,41.872187,-87.661501,"(41.872187, -87.661501)",Harrison & Loomis,0.134892,Racine,0.278191,0
1417,54,Ogden Ave & Chicago Ave,19,19,In Service,41.896362,-87.654061,"(41.896362458, -87.6540612729)",Chicago & Milwaukee (Blue Line),0.053932,Chicago,0.062626,0


In [6]:
bus_weight = 0.5
train_weight = 0.3
dist_s1_weight = 0.8
dist_s2_weight = 0.5
dist_s3_weight = 0.3

def find_efficiency(station_data):
    for i in range(len(station_data)):
        efficiency = station_data.loc[i]["nearest_bus_station_distance"] * bus_weight
        efficiency = efficiency + station_data.loc[i]["nearest_train_station_distance"] * train_weight
    

        dist = []
        for j in range(len(station_data)):
            if i != j:
                dist.append(geopy.distance.geodesic(literal_eval(station_data.loc[i]["Location"]), literal_eval(station_data.loc[j]["Location"])).miles)
        dist.sort()

        efficiency = efficiency + dist[0]*dist_s1_weight
        efficiency = efficiency + dist[1]*dist_s2_weight
        efficiency = efficiency + dist[2]*dist_s3_weight

        station_data.loc[i,"efficiency"] = efficiency

In [7]:
find_efficiency(stations)

In [8]:
stations

,ID,Station Name,Total Docks,Docks in Service,Status,Latitude,Longitude,Location,nearest_bus_station_name,nearest_bus_station_distance,nearest_train_station_name,nearest_train_station_distance,efficiency
0,1594046383808271024,Troy St & Jackson Blvd,9,9,In Service,41.877505,-87.704850,"(41.877505, -87.70485)",Jackson & Kedzie,0.050582,Kedzie-Homan,0.226695,0.636945
1,641,Central Park Ave & Bloomingdale Ave,11,11,In Service,41.914166,-87.716755,"(41.914166, -87.716755)",Armitage & Central Park,0.212447,Logan Square,1.153785,1.037167
2,1683527931525155814,Public Rack - Cornell Ave & 87th Pl,1,1,In Service,41.736881,-87.583146,"(41.73688127, -87.58314552)",87th Street & East End,0.012684,87th,2.150201,0.730491
3,367,Racine Ave & 35th St,15,15,In Service,41.830689,-87.656211,"(41.83068856472101, -87.65621066093445)",35th Street & Racine,0.031093,Ashland,0.753699,1.060780
4,1673852313397164648,Public Rack - Francisco Ave & Touhy Ave,2,2,In Service,42.011865,-87.701317,"(42.011865, -87.701317)",Touhy & Francisco,0.044888,Howard,1.544120,0.571182
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414,1715846705501012844,Public Rack - Lawndale Ave & Roosevelt,2,2,In Service,41.866225,-87.717918,"(41.866225, -87.717918)",Roosevelt & Lawndale,0.003483,Pulaski,0.657426,0.400917
1415,1715846718385914742,Public Rack - Ashland Ave & 74th St,2,2,In Service,41.759735,-87.663736,"(41.759735, -87.663736)",74th Street & Ashland,0.021196,Ashland/63rd,1.319123,0.551977
1416,320,Loomis St & Lexington St,19,19,In Service,41.872187,-87.661501,"(41.872187, -87.661501)",Harrison & Loomis,0.134892,Racine,0.278191,0.605246
1417,54,Ogden Ave & Chicago Ave,19,19,In Service,41.896362,-87.654061,"(41.896362458, -87.6540612729)",Chicago & Milwaukee (Blue Line),0.053932,Chicago,0.062626,0.432207


In [9]:
df_min_max_scaled = stations.copy()
column = 'efficiency'
df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())

In [10]:
df_min_max_scaled

,ID,Station Name,Total Docks,Docks in Service,Status,Latitude,Longitude,Location,nearest_bus_station_name,nearest_bus_station_distance,nearest_train_station_name,nearest_train_station_distance,efficiency
0,1594046383808271024,Troy St & Jackson Blvd,9,9,In Service,41.877505,-87.704850,"(41.877505, -87.70485)",Jackson & Kedzie,0.050582,Kedzie-Homan,0.226695,0.163976
1,641,Central Park Ave & Bloomingdale Ave,11,11,In Service,41.914166,-87.716755,"(41.914166, -87.716755)",Armitage & Central Park,0.212447,Logan Square,1.153785,0.279250
2,1683527931525155814,Public Rack - Cornell Ave & 87th Pl,1,1,In Service,41.736881,-87.583146,"(41.73688127, -87.58314552)",87th Street & East End,0.012684,87th,2.150201,0.190920
3,367,Racine Ave & 35th St,15,15,In Service,41.830689,-87.656211,"(41.83068856472101, -87.65621066093445)",35th Street & Racine,0.031093,Ashland,0.753699,0.286051
4,1673852313397164648,Public Rack - Francisco Ave & Touhy Ave,2,2,In Service,42.011865,-87.701317,"(42.011865, -87.701317)",Touhy & Francisco,0.044888,Howard,1.544120,0.145035
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414,1715846705501012844,Public Rack - Lawndale Ave & Roosevelt,2,2,In Service,41.866225,-87.717918,"(41.866225, -87.717918)",Roosevelt & Lawndale,0.003483,Pulaski,0.657426,0.095994
1415,1715846718385914742,Public Rack - Ashland Ave & 74th St,2,2,In Service,41.759735,-87.663736,"(41.759735, -87.663736)",74th Street & Ashland,0.021196,Ashland/63rd,1.319123,0.139503
1416,320,Loomis St & Lexington St,19,19,In Service,41.872187,-87.661501,"(41.872187, -87.661501)",Harrison & Loomis,0.134892,Racine,0.278191,0.154846
1417,54,Ogden Ave & Chicago Ave,19,19,In Service,41.896362,-87.654061,"(41.896362458, -87.6540612729)",Chicago & Milwaukee (Blue Line),0.053932,Chicago,0.062626,0.105006


In [11]:
stations.to_csv('stations_efficiency.csv')

In [13]:
station_community_mapping = pd.read_csv("StationCommunityMapping.csv")
station_community_mapping.head()

,Unnamed: 0,ID,Station Name,Total Docks,Docks in Service,Status,Latitude,Longitude,Location,community_area
0,0,1594046383808271024,Troy St & Jackson Blvd,9,9,In Service,41.877505,-87.704850,"(41.877505, -87.70485)",EAST_GARFIELD_PARK
1,1,641,Central Park Ave & Bloomingdale Ave,11,11,In Service,41.914166,-87.716755,"(41.914166, -87.716755)",LOGAN_SQUARE
2,2,1683527931525155814,Public Rack - Cornell Ave & 87th Pl,1,1,In Service,41.736881,-87.583146,"(41.73688127, -87.58314552)",CALUMET_HEIGHTS
3,3,367,Racine Ave & 35th St,15,15,In Service,41.830689,-87.656211,"(41.83068856472101, -87.65621066093445)",BRIDGEPORT
4,4,1673852313397164648,Public Rack - Francisco Ave & Touhy Ave,2,2,In Service,42.011865,-87.701317,"(42.011865, -87.701317)",WEST_RIDGE


In [14]:
df_min_max_scaled['community_area'] = station_community_mapping['community_area']

In [15]:
df_min_max_scaled.head()

,ID,Station Name,Total Docks,Docks in Service,Status,Latitude,Longitude,Location,nearest_bus_station_name,nearest_bus_station_distance,nearest_train_station_name,nearest_train_station_distance,efficiency,community_area
0,1594046383808271024,Troy St & Jackson Blvd,9,9,In Service,41.877505,-87.704850,"(41.877505, -87.70485)",Jackson & Kedzie,0.050582,Kedzie-Homan,0.226695,0.163976,EAST_GARFIELD_PARK
1,641,Central Park Ave & Bloomingdale Ave,11,11,In Service,41.914166,-87.716755,"(41.914166, -87.716755)",Armitage & Central Park,0.212447,Logan Square,1.153785,0.279250,LOGAN_SQUARE
2,1683527931525155814,Public Rack - Cornell Ave & 87th Pl,1,1,In Service,41.736881,-87.583146,"(41.73688127, -87.58314552)",87th Street & East End,0.012684,87th,2.150201,0.190920,CALUMET_HEIGHTS
3,367,Racine Ave & 35th St,15,15,In Service,41.830689,-87.656211,"(41.83068856472101, -87.65621066093445)",35th Street & Racine,0.031093,Ashland,0.753699,0.286051,BRIDGEPORT
4,1673852313397164648,Public Rack - Francisco Ave & Touhy Ave,2,2,In Service,42.011865,-87.701317,"(42.011865, -87.701317)",Touhy & Francisco,0.044888,Howard,1.544120,0.145035,WEST_RIDGE


In [16]:
df_min_max_scaled["demandSupplyRatio"] = df_min_max_scaled['efficiency']*10

In [17]:
df_min_max_scaled.head()

,ID,Station Name,Total Docks,Docks in Service,Status,Latitude,Longitude,Location,nearest_bus_station_name,nearest_bus_station_distance,nearest_train_station_name,nearest_train_station_distance,efficiency,community_area,demandSupplyRatio
0,1594046383808271024,Troy St & Jackson Blvd,9,9,In Service,41.877505,-87.704850,"(41.877505, -87.70485)",Jackson & Kedzie,0.050582,Kedzie-Homan,0.226695,0.163976,EAST_GARFIELD_PARK,1.639761
1,641,Central Park Ave & Bloomingdale Ave,11,11,In Service,41.914166,-87.716755,"(41.914166, -87.716755)",Armitage & Central Park,0.212447,Logan Square,1.153785,0.279250,LOGAN_SQUARE,2.792503
2,1683527931525155814,Public Rack - Cornell Ave & 87th Pl,1,1,In Service,41.736881,-87.583146,"(41.73688127, -87.58314552)",87th Street & East End,0.012684,87th,2.150201,0.190920,CALUMET_HEIGHTS,1.909197
3,367,Racine Ave & 35th St,15,15,In Service,41.830689,-87.656211,"(41.83068856472101, -87.65621066093445)",35th Street & Racine,0.031093,Ashland,0.753699,0.286051,BRIDGEPORT,2.860515
4,1673852313397164648,Public Rack - Francisco Ave & Touhy Ave,2,2,In Service,42.011865,-87.701317,"(42.011865, -87.701317)",Touhy & Francisco,0.044888,Howard,1.544120,0.145035,WEST_RIDGE,1.450346


In [ ]:
bus_weight = 0.2
train_weight = 0.1
dist_s1_weight = 0.4
dist_s2_weight = 0.2
dist_s3_weight = 0.1
demand_supply_ration = 0.8

def find_efficiency(station_data):
    for i in range(len(station_data)):
        efficiency = station_data.loc[i]["nearest_bus_station_distance"] * bus_weight
        efficiency = efficiency + station_data.loc[i]["nearest_train_station_distance"] * train_weight
    

        dist = []
        for j in range(len(station_data)):
            if i != j:
                dist.append(geopy.distance.geodesic(literal_eval(station_data.loc[i]["Location"]), literal_eval(station_data.loc[j]["Location"])).miles)
        dist.sort()

        efficiency = efficiency + dist[0]*dist_s1_weight
        efficiency = efficiency + dist[1]*dist_s2_weight
        efficiency = efficiency + dist[2]*dist_s3_weight
        efficiency = efficiency + station_data.loc[i]["demandSupplyRatio"]*demand_supply_ration

        station_data.loc[i,"efficiency"] = efficiency

In [18]:
find_efficiency(df_min_max_scaled)

In [19]:
df_min_max_scaled.head()

,ID,Station Name,Total Docks,Docks in Service,Status,Latitude,Longitude,Location,nearest_bus_station_name,nearest_bus_station_distance,nearest_train_station_name,nearest_train_station_distance,efficiency,community_area,demandSupplyRatio
0,1594046383808271024,Troy St & Jackson Blvd,9,9,In Service,41.877505,-87.704850,"(41.877505, -87.70485)",Jackson & Kedzie,0.050582,Kedzie-Homan,0.226695,0.636945,EAST_GARFIELD_PARK,1.639761
1,641,Central Park Ave & Bloomingdale Ave,11,11,In Service,41.914166,-87.716755,"(41.914166, -87.716755)",Armitage & Central Park,0.212447,Logan Square,1.153785,1.037167,LOGAN_SQUARE,2.792503
2,1683527931525155814,Public Rack - Cornell Ave & 87th Pl,1,1,In Service,41.736881,-87.583146,"(41.73688127, -87.58314552)",87th Street & East End,0.012684,87th,2.150201,0.730491,CALUMET_HEIGHTS,1.909197
3,367,Racine Ave & 35th St,15,15,In Service,41.830689,-87.656211,"(41.83068856472101, -87.65621066093445)",35th Street & Racine,0.031093,Ashland,0.753699,1.060780,BRIDGEPORT,2.860515
4,1673852313397164648,Public Rack - Francisco Ave & Touhy Ave,2,2,In Service,42.011865,-87.701317,"(42.011865, -87.701317)",Touhy & Francisco,0.044888,Howard,1.544120,0.571182,WEST_RIDGE,1.450346


In [31]:
newDf = pd.DataFrame(df_min_max_scaled.groupby("community_area")["efficiency"].mean()).reset_index(drop=False)


In [34]:
newDf.sort_values(["efficiency"],ascending=True)

,community_area,efficiency
37,LOOP,0.336950
63,WASHINGTON_PARK,0.345812
43,NEAR_NORTH_SIDE,0.374059
66,WEST_GARFIELD_PARK,0.404651
45,NEAR_WEST_SIDE,0.443565
...,...,...
57,SOUTH_CHICAGO,1.605672
42,MOUNT_GREENWOOD,1.608150
19,EAST_SIDE,1.715258
54,RIVERDALE,1.824135


community_area
ALBANY_PARK       0.537085
ARCHER_HEIGHTS           d
ARMOUR_SQUARE     0.629914
ASHBURN           1.260986
AUBURN_GRESHAM    0.856962
                    ...   
WEST_LAWN          0.55352
WEST_PULLMAN      1.457296
WEST_RIDGE        0.799921
WEST_TOWN         0.579211
WOODLAWN          0.647985
Name: efficiency, Length: 72, dtype: object